In [47]:
import numpy as np
import sqlite3
from typing import List, Optional, Tuple

# Dev Data Manager for Lapin object

Example data set

In [16]:
data = [
    ("usagi", ["CX12", "CZ23", "X1"], [1.23, 2.34, 3.45], 1),
    ("usausa", ["CX23", "CZ31", "X2"], [4.23, 1.34, 2.45], 2),
    ("ucchi", ["Z1", "CZ12", "X3"], [0.23, 2.34, -2.45], 1)
]

## Model class

In [17]:
class Lapin:

    def __init__(
            self,
            id_: int,
            name: str,
            genome: List[str],
            points: List[float],
            user_id: int,
            date: str
    ):
        self.id = id_
        self.name = name
        self.genome = genome
        self.points = points
        self.user = user_id
        self.date = date

    def __str__(self):
        ret = f"Lapin[id={self.id} name={self.name} user={self.user}]"
        return ret

## Make DB

In [18]:
# config
cf = dict(
    name="lapin.db"
)


In [19]:
def make_table_lapin(dbname: str):
    query_fields = [
        "id integer primary key",
        "name string",
        "genome string",
        "points string",
        "user int",
        "date timestamp default (datetime(current_timestamp,'localtime'))"
    ]
    query = f"create table if not exists lapin({', '.join(query_fields)})"
    with sqlite3.connect(dbname) as conn:
        cur = conn.cursor()
        cur.execute(query)
        conn.commit()


In [20]:
make_table_lapin(cf["name"])


In [48]:
class LapinDataManager:

    def __init__(self, dbname: str):
        self.dbname = dbname
        make_table_lapin(dbname)

    def load_lapin(
            self,
            id_: int = None,
            name: str = None,
            user_id: int = None,
    ) -> List[Lapin]:
        conditions = []
        if id_ is not None:
            conditions.append(f"id = '{id_}'")
        if name is not None:
            conditions.append(f"name = '{name}'")
        if user_id is not None:
            conditions.append(f"user = '{user_id}'")

        query = "select * from lapin"
        if len(conditions) > 0:
            str_conditions = " and ".join(conditions)
            query = query + " where " + str_conditions

        with sqlite3.connect(self.dbname) as conn:
            cur = conn.cursor()
            cur.execute(query)
            lapins = [Lapin(*self.decode_data(row)) for row in cur]

        return lapins

    @staticmethod
    def decode_data(row: tuple) -> Tuple:
        id_, name, str_genome, str_points, user_id, date = row
        genome = str_genome.split(":")
        points = str_points.split(":")
        points = [float(p) / 100 for p in points]
        return id_, name, genome, points, user_id, date

    def store_lapin(
            self,
            name: str,
            genome: List[str],
            points: List[float],
            user_id: int,
    ) -> List[Lapin]:
        points = np.asarray(points) * 100
        str_points = ":".join([str(p) for p in points.astype(int)])
        str_genome = ":".join(genome)

        query_base = "insert into lapin (name, genome, points, user) values "
        query_data = "('" + "', '".join([name, str_genome, str_points, str(user_id)]) + "')"
        query = query_base + query_data

        with sqlite3.connect(self.dbname) as conn:
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()

    def create(
            self,
            name: str,
            genome: List[str],
            points: List[float],
            user_id: int,
    ):
        self.store_lapin(name, genome, points, user_id)
        lapin = self.load_lapin(name=name, user_id=user_id)[0]
        return lapin